# Script for Pre-processing Dataset 

This script is created with intention to document steps in preparing dataset to do forest loss analysis. There are 3 outcomes generated from this script. They are:
1) Terrestrial Mammals Area of Habitat Based on IUCN Red List Category listed as Critically Endangered, Endangered, and Vulnerable (shp format)
2) Birds Area of Habitat from Birdllife with Red List Category from IUCN (shp format)
3) Planted Trees 2017 and 2020 (Following WRI definition) (tif format)

## Mammals

Dataset used here:
1) Terrestrial mammal area of habitat (IUCN) available at https://www.iucnredlist.org/resources/spatial-data-download (1 July 2025)
3) Bird Species' Area of Habitat (AOH) maps  requested from https://datazone.birdlife.org/contact-us/request-our-data (1 July 2025)
4) All species' habitat, minimum and maximum altitude are collected through IUCN Redlist Webpage by entering species' name one by one (July 2025), saved here with mammals_database.xls & bird_database.xls
5) Bird' IUCN redlist category available at https://datazone.birdlife.org/about-our-science/taxonomy (25 July 2025) named here as bird_iucn_taxonomy.xls file
6) Spatial Database of Planted Forest (WRI) https://www.wri.org/research/spatial-database-planted-trees-sdpt-version-2 (July 2025) downloaded from 
   https://gfw-files.s3.amazonaws.com/plantations/SDPT_v2.1/sdpt_v21_v09152024_public.gdb.zip, I export the shp using QGIS (I try to be consistent in using python but always fail when opening using fiona, using Rstudio took a very long, using Arcgis got warning issues)
8) Borneo Boundary (gadm41_Borneo_IDN_1) downloaded at https://gadm.org/ simplify vertex at 200m using ArcgisPro, explode and delete small-scattered islands except :
   - Maya Karimata island -1.1114594, 109.5900927 & -0.807859, 109.441845
   - Laut Selatan island -3.67704706, 116.1493258
   - Sebuku island -3.515927, 116.3694682
   - Mahakam Delta -0.663428, 117.401863
   - and Bulungan River Estuary 3.027885, 117.468919

In [1]:
import numpy as np
import matplotlib as plt
import geopandas as gpd
import os

In [3]:
os.getcwd()

'C:\\Users\\nooriza maharani\\Documents\\Dissertation'

In [3]:
borneo_boundary = gpd.read_file('gadm41_Borneo.shp')

In [4]:
mammal = gpd.read_file('MAMMALS_TERRESTRIAL_ONLY.shp')

In [5]:
bird = gpd.read_file('BOTW_2024_2.gpkg')

C:\Anaconda\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'BOTW_2024_2.gpkg': 'all_species' (default), 'main_BL_HBW_Checklist_V9'. Specify layer parameter to avoid this warning.
  result = read_func(


In [6]:
import fiona

# List all layers in the GPKG file
layers = fiona.listlayers("BOTW_2024_2.gpkg")
print(layers)

['all_species', 'main_BL_HBW_Checklist_V9']


In [7]:
borneo_boundary.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
mammal.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
bird.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Clipping Mammals with Borneo Boundary

In [11]:
# Cek geometri yang tidak valid
invalid = mammal[~mammal.is_valid]
print(invalid)

      id_no             sci_name  presence  origin  seasonal      compiler  \
2535   9760  Helarctos malayanus         5       1         1  Graham Usher   
9831  41772         Sus barbatus         1       1         1          IUCN   

      yrcompiled                                           citation  \
2535        2017               Scotson, L. and Fredriksson, G. 2016   
9831        2017  IUCN (International Union for Conservation of ...   

     subspecies subpop  ... marine terrestria freshwater  SHAPE_Leng  \
2535       None   None  ...  false       true      false  298.047663   
9831   barbatus   None  ...  false       true      false  203.352979   

      SHAPE_Area           area  \
2535   28.227425  347094.755252   
9831   29.450292  362205.059335   

                                                habitat altitude max_alt  \
2535          Forest, Shrubland, Artificial/Terrestrial      1.0  3000.0   
9831  Forest, Wetlands (inland), Marine Neritic, Mar...      0.0     0.0   


In [12]:
# Select animals which presence is not extinct and resident through the year
mammals_filtered = mammal[(mammal['presence'] != 5) & (mammal['seasonal'] == 1)].copy()

In [13]:
len(mammals_filtered)

12450

In [ ]:
from shapely.validation import make_valid

mammals_filtered["geometry"] = mammals_filtered["geometry"].apply(make_valid)
borneo_boundary["geometry"] = borneo_boundary["geometry"].apply(make_valid)

In [ ]:
mammals_borneo = gpd.clip(mammals_filtered, borneo_boundary)

In [ ]:
# Plot the clipped layer
mammals_borneo.plot(figsize=(10, 10), edgecolor='black', cmap='Set2')
plt.title("Clipped Mammals Layer")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
# Select animals where yrcompiled > 2000 and category as CR, VU, EN
mammals_threatened = mammals_borneo[
    (mammals_borneo['category'].isin(['VU', 'CR', 'EN'])) &
    (mammals_borneo['yrcompiled'] > 2000)
].copy()

In [ ]:
len(mammals_threatened)

In [ ]:
# Reproject to Southeast Asia Albers Equal Area (ESRI:102025) in square metres
mammals_borneo_proj = mammals_threatened.to_crs("ESRI:102025")

# Calculate area in square meters
mammals_borneo_proj['area_m2'] = mammals_borneo_proj.geometry.area
mammals_borneo_proj['area_km2'] = mammals_borneo_proj['area_m2'] / 1_000_000
# Remove small areas possibly the result of clipping process
mammals_borneo_proj = mammals_borneo_proj[mammals_borneo_proj['area_km2'] >= 80]

In [ ]:
print(mammals_borneo_proj.columns.to_list())

In [ ]:
# Sort by SHAPE_Area in descending order
mammals_sorted = mammals_borneo_proj.sort_values(by='area_km2', ascending=False)

# Display selected columns
#print(mammals_sorted[['sci_name', 'category', 'area_km2']])

In [ ]:
# Merged duplicate species name
# Sort to keep largest area per species
sorted_gdf = mammals_borneo_proj.sort_values('area_km2', ascending=False)

# Group and pick first (largest area)
mammals_merged = (
    sorted_gdf
    .groupby('sci_name', as_index=False)
    .agg({
        'category': lambda x: ', '.join(sorted(set(x))),
        'yrcompiled': lambda x: ', '.join(map(str, sorted(set(x)))),
        'area_km2': 'first',
        'geometry': 'first'
    })
)

# Convert to GeoDataFrame again
import geopandas as gpd
mammals_merged = gpd.GeoDataFrame(mammals_merged, geometry='geometry', crs=mammals_borneo_proj.crs)

In [ ]:
# Sort by SHAPE_Area in descending order
mammals_sort = mammals_merged.sort_values(by='sci_name', ascending=False)

# Display selected columns
#print(mammals_sort[['sci_name', 'category', 'yrcompiled', 'area_km2']])

In [ ]:
len(mammals_merged)

In [ ]:
iucn_data = pd.read_excel('mammals_database.xls')

In [ ]:
mammal_aoh = mammals_merged.merge(iucn_data, on = 'sci_name', how = 'left')

In [ ]:
mammal_aoh= mammal_aoh.dropna(subset=['max_alt'])

In [ ]:
len(mammal_aoh)

In [ ]:
#mammals_merged.to_file("mammals_merged.gpkg", driver = "GPKG")

In [ ]:
mammal_aoh.to_file("mammal_aoh.shp")

### Clipping Birds with Borneo Boundary

In [ ]:
# Invalid Geometry
invalid_bird = bird[~bird.is_valid]
print(invalid1)

In [ ]:
bird_borneo = gpd.clip(bird, borneo_boundary)

In [ ]:
len(bird_borneo)

In [ ]:
# Plot the clipped layer
bird_borneo.plot(figsize=(10, 10), edgecolor='black', cmap='Set2')
plt.title("Clipped bird Layer")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
# Select animals which presence is not extinct and resident through the year
bird_borneo_f = bird_borneo[(bird_borneo['presence'] != 5) & (bird_borneo['seasonal'] == 1) & (bird_borneo['yrcompiled'] > 2000)].copy()

In [ ]:
iucn_data_bird = pd.read_excel('birds_iucn_taxonomy.xls')

In [ ]:
bird_aoh = bird_borneo_f.merge(iucn_data_bird, on = 'sci_name', how = 'left')

In [ ]:
birds_threatened_aoh = bird_aoh[bird_aoh['iucn_category'].isin(['VU', 'CR', 'EN'])].copy()

In [ ]:
len(birds_threatened_aoh)

In [ ]:
# Reproject to Southeast Asia Albers Equal Area (ESRI:102025) in square metres
bird_proj = birds_threatened_aoh.to_crs("ESRI:102025")

# Calculate area in square meters
bird_proj['area_m2'] = bird_proj.geometry.area
bird_proj['area_km2'] = bird_proj['area_m2'] / 1_000_000

In [ ]:
# Merged duplicate species name
# Sort to keep largest area per species
bird_gdf = bird_proj.sort_values('area_km2', ascending=False)

# Group and pick first (largest area)
bird_merged = (
    bird_gdf
    .groupby('sci_name', as_index=False)
    .agg({
        'iucn_category': lambda x: ', '.join(sorted(set(x))),
        'yrcompiled': lambda x: ', '.join(map(str, sorted(set(x)))),
        'area_km2': 'first',
        'geometry': 'first'
    })
)

# Convert to GeoDataFrame again
import geopandas as gpd
bird_merged = gpd.GeoDataFrame(bird_merged, geometry='geometry', crs=bird_proj.crs)

In [ ]:
# Sort by SHAPE_Area in descending order
bird_sort = bird_merged.sort_values(by='area_km2', ascending=False)

# Display selected columns
print(bird_sort[['sci_name', 'iucn_category', 'yrcompiled', 'area_km2']])

In [ ]:
altitude = pd.read_excel('birds_database.xls')

In [ ]:
aoh_bird = bird_merged.merge(altitude, on = 'sci_name', how = 'left')

In [ ]:
print(aoh_bird)

In [ ]:
aoh_bird= aoh_bird.dropna(subset=['max_alt'])
len(aoh_bird)

In [ ]:
aoh_bird.to_file("aoh_bird.shp")

## Working with Planted Forest Dataset

What I do here:
1) Note: There are several invalid geomtries for this SDPT dataset. I try to use shapely and initially wanted to do clipping in python, but the operation takes too long, so I used GIS desktop software to clip the data into Borneo Boundary
2) Perform filtering trees that are planted before and equal to year 2017 as a planted trees for 2017 and use the SDPT V21 official dataset in 2020 as planted trees for 2020.
3) I rasterize them here with output of 30 x 30 pixels, matching Hansen Forest Loss Dataset and compressed them before uploaded into GEE asset

In [4]:
os.getcwd()

'C:\\Users\\nooriza maharani\\Documents\\Dissertation'

In [5]:
#sdpt = gpd.read_file("sdpt_idn_plant_v21.shp")

In [14]:
# Invalid Geometry
invalid_sdpt = sdpt[~sdpt.is_valid]
print(invalid_sdpt)

        OBJECTID final_id iso3 iso2    country  \
88          89.0   IDN_38  IDN   ID  Indonesia   
89          90.0   IDN_38  IDN   ID  Indonesia   
102        103.0   IDN_38  IDN   ID  Indonesia   
105        106.0   IDN_38  IDN   ID  Indonesia   
107        108.0   IDN_38  IDN   ID  Indonesia   
...          ...      ...  ...  ...        ...   
953347  953348.0  IDN_203  IDN   ID  Indonesia   
953355  953356.0  IDN_203  IDN   ID  Indonesia   
953359  953360.0  IDN_203  IDN   ID  Indonesia   
953361  953362.0  IDN_203  IDN   ID  Indonesia   
953365  953366.0  IDN_203  IDN   ID  Indonesia   

                                               originalNa  originalCo  \
88      Large industrial plantation, Oil palm, transpa...          38   
89      Large industrial plantation, Oil palm, transpa...          38   
102     Large industrial plantation, Oil palm, transpa...          38   
105     Large industrial plantation, Oil palm, transpa...          38   
107     Large industrial plantatio

In [7]:
# RUn this but takes too long, so I prefer using QGIS to clip the planted forest
#from shapely.validation import make_valid
#sdpt["geometry"] = sdpt["geometry"].apply(make_valid)


KeyboardInterrupt



In [9]:
borneo_sdpt = gpd.read_file('planted_crops.shp')

In [11]:
borneo_sdpt.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
# Reproject to Southeast Asia Albers Equal Area (ESRI:102025) in square metres
borneo_sdpt_proj = borneo_sdpt.to_crs("ESRI:102025")

In [18]:
borneo_sdpt_proj.columns

Index(['FID', 'OBJECTID', 'final_id', 'iso3', 'iso2', 'country', 'originalNa',
       'originalCo', 'finalCode', 'vernacName', 'sciName', 'simpleName',
       'sciName1', 'sciName2', 'sciName3', 'sciName4', 'sciName5', 'sciName6',
       'sciName7', 'sciName8', 'sciName9', 'sciName10', 'sciName11',
       'sciName12', 'sciName13', 'simpleType', 'leafStatus', 'leafType',
       'woodType', 'sizeCatego', 'ownership', 'reference', 'imageryYea',
       'method', 'removalFac', 'removalF_1', 'removalF_2', 'removalF_3',
       'removalF_4', 'removalF_5', 'removalF_6', 'removalF_7', 'plantedYea',
       'Shape_Leng', 'Shape_Area', 'geometry'],
      dtype='object')

In [20]:
len(borneo_sdpt_proj)

706313

In [24]:
sdpt_2017 = borneo_sdpt_proj[borneo_sdpt_proj['plantedYea'] <= 2017]

In [26]:
len(sdpt_2017)

646733

In [28]:
#sdpt_2017.to_file('sdpt_2017.shp')
#borneo_sdpt_proj.to_file('sdpt_2020.shp')

In [32]:
pip install geopandas rasterio rasterio.features

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement rasterio.features (from versions: none)
ERROR: No matching distribution found for rasterio.features


In [34]:
# Changing vector file into raster in 30x30
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import numpy as np

# Define raster resolution and bounds
resolution = 30  # in the units of  CRS (meters)
bounds = sdpt_2017.total_bounds  # [minx, miny, maxx, maxy]

width = int((bounds[2] - bounds[0]) / resolution)
height = int((bounds[3] - bounds[1]) / resolution)

transform = from_bounds(*bounds, width=width, height=height)

# Create (geometry, value) pairs — all values set to 1 (plantation)
shapes = [(geom, 1) for geom in sdpt_2017.geometry]

# Rasterize
raster = rasterize(
    shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,  # 0 = no plantation
    dtype='uint8'
)

# Save as GeoTIFF
with rasterio.open(
    'plantation_2017.tif',
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype='uint8',
    crs=sdpt_2017.crs,
    transform=transform,
) as dst:
    dst.write(raster, 1)


In [38]:
# Changing vector file into raster in 30x30
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import numpy as np

# Define raster resolution and bounds
resolution = 30  # in the units of your CRS (e.g., meters)
bounds = borneo_sdpt_proj.total_bounds  # [minx, miny, maxx, maxy]

width = int((bounds[2] - bounds[0]) / resolution)
height = int((bounds[3] - bounds[1]) / resolution)

transform = from_bounds(*bounds, width=width, height=height)

# Create (geometry, value) pairs — all values set to 1 (plantation)
shapes = [(geom, 1) for geom in borneo_sdpt_proj.geometry]

# Rasterize
raster = rasterize(
    shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,  # 0 = no plantation
    dtype='uint8'
)

# Save as GeoTIFF
with rasterio.open(
    'plantation_2020.tif',
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype='uint8',
    crs=sdpt_2017.crs,
    transform=transform,
) as dst:
    dst.write(raster, 1)


In [42]:
# Compress the geotiff output
!gdal_translate -co COMPRESS=LZW -co TILED=YES plantation_2017.tif output_compressed.tif

Input file size is 40582, 29940
0...10...20...30...40...50...60...70...80...90...100 - done in 00:00:11.


In [44]:
# Check whether the original raster similar in shape and value with compressed raster
with rasterio.open("plantation_2017.tif") as src1, rasterio.open("output_compressed.tif") as src2:
    data1 = src1.read(1)
    data2 = src2.read(1)

    print("Same shape?", data1.shape == data2.shape)
    print("All values equal?", np.array_equal(data1, data2))


Same shape? True
All values equal? True


In [49]:
# Compress the geotiff output
!gdal_translate -co COMPRESS=LZW -co TILED=YES plantation_2020.tif output_compressed_2020.tif

Input file size is 40582, 29940
0...10...20...30...40...50...60...70...80...90...100 - done in 00:00:12.


In [59]:
# Check whether the original raster similar in shape and value with compressed raster
with rasterio.open("plantation_2020.tif") as src1, rasterio.open("output_compressed_2020.tif") as src2:
    data1 = src1.read(1)
    data2 = src2.read(1)

    print("Same shape?", data1.shape == data2.shape)
    print("All values equal?", np.array_equal(data1, data2))


Same shape? True
All values equal? True
